In [18]:
from flytekit.configuration import platform
def print_console_url(exc):
    print("http://{}/console/projects/{}/domains/{}/executions/{}".format(platform.URL.get(), exc.id.project, exc.id.domain, exc.id.name))

# Build, Register & Launch

### 1) Build a docker image
To ensure repeatablity, we recommend building a docker image with a new version to reflect any code change.

Note: You need to have push permissions on a docker hub repo (or any other docker registry) to execute the docker push command.

In [25]:
from os import environ
environ["version"]="12345"
!docker build -t docker.io/lyft/flytesnacks:datacouncil-$version .
!docker push docker.io/lyft/flytesnacks:datacouncil-$version

Sending build context to Docker daemon  198.7kB
Step 1/19 : FROM ubuntu:bionic
 ---> 775349758637
Step 2/19 : WORKDIR /root
 ---> Using cache
 ---> 9e71f2c2950c
Step 3/19 : ENV VENV /opt/venv
 ---> Using cache
 ---> 7605b00f67e3
Step 4/19 : ENV LANG C.UTF-8
 ---> Using cache
 ---> 197d97963594
Step 5/19 : ENV LC_ALL C.UTF-8
 ---> Using cache
 ---> f424c74f8294
Step 6/19 : ENV PYTHONPATH /root
 ---> Using cache
 ---> ce39083a6d7c
Step 7/19 : RUN :     && apt-get update && apt-get install -y python3.6 python3.6-venv python3.6-dev make build-essential libssl-dev python3-venv python3-pip curl
 ---> Using cache
 ---> fbbef8af0eb2
Step 8/19 : RUN apt-get update && apt-get install -y libsm6 libxext6 libxrender-dev
 ---> Using cache
 ---> 61cfd96195b8
Step 9/19 : RUN pip3 install awscli
 ---> Using cache
 ---> 9ec5fd79ae76
Step 10/19 : RUN python3.6 -m venv ${VENV}
 ---> Using cache
 ---> c9f5e3d2048a
Step 11/19 : RUN ${VENV}/bin/pip install wheel
 ---> Using cache
 ---> 14c2e932b36c
Step 12/1

### 2) Register all tasks and workflows in this new image
FlyteKit can discover all tasks, workflows and launchplans in a given python package(s).

In [26]:
# Register existing tasks/workflows with the system
from flytekit.clis.sdk_in_container.register import register_all, register_tasks_only
from os import environ
from flytekit.configuration import set_flyte_config_file

environ["FLYTE_INTERNAL_IMAGE"] = "docker.io/lyft/flytesnacks:datacouncil-{}".format(environ["version"])
set_flyte_config_file("/Users/hamabuelfutuh/.flyte/config")
environ["FLYTE_INTERNAL_CONFIGURATION_PATH"] = "/Users/hamabuelfutuh/.flyte/config"

register_all(project="flytesnacks", domain="development", pkgs=["workflows"], version=environ["version"], test=False)

Running task, workflow, and launch plan registration for flytesnacks, development, ['workflows'] with version 12345
Registering Task:                workflows.recipe_1.inner.inner_task
Registering Workflow:            workflows.recipe_1.inner.IdentityWorkflow
Registering Launch Plan:         workflows.recipe_1.inner.IdentityWorkflow
Registering Launch Plan:         workflows.recipe_1.inner.secondary_sibling_identity_lp
Registering Task:                workflows.recipe_1.outer.lp_yield_task
Registering Workflow:            workflows.recipe_1.outer.DynamicLaunchPlanCaller
Registering Launch Plan:         workflows.recipe_1.outer.DynamicLaunchPlanCaller
Registering Task:                workflows.recipe_1.outer.sub_wf_yield_task
Registering Workflow:            workflows.recipe_1.outer.DynamicSubWorkflowCaller
Registering Launch Plan:         workflows.recipe_1.outer.DynamicSubWorkflowCaller
Registering Launch Plan:         workflows.recipe_1.outer.id_lp
Registering Workflow:            wo

### 3) Launch a workflow

In [32]:
from workflows.recipe_1.inner import IdentityWorkflow
from os import environ

IdentityWorkflow_lp = IdentityWorkflow.create_launch_plan()
IdentityWorkflow_lp.register(name="IdentityWorkflow", project="flytesnacks", domain="development", version=environ["version"])
exec = IdentityWorkflow_lp.launch(project="flytesnacks", domain="development", inputs={})
print_console_url(exec)

http://flyte.lyft.net/console/projects/flytesnacks/domains/development/executions/f45f1d0918283471f838


In [33]:
print("Waiting for workflow execution to complete")
exec.wait_for_completion()
print("Done!")
print()
print("Output: {}".format(exec.outputs["task_output"]))

Waiting for workflow execution to complete
Done!

Output: 5
